In [3]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


################################################################################
## Music Stuff
################################################################################

## General
from discogsBase import discogs
from multiArtist import multiartist
from myMusicDBMap import myMusicDBMap

## Discogs
from artistDC import artistDC
from artistsDC import artistsDC
from discogsUtils import discogsUtils
from albums import albums

## AllMusic
from artistAM import artistAM
from artistsAM import artistsAM
from discogsUtils import allmusicUtils
from artistsAM import artistsAM

## MusicBrainz
from artistMB import artistMB
from artistsMB import artistsMB
from discogsUtils import musicbrainzUtils

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils

## DatPiff
from artistDP import artistDP
from artistsDP import artistsDP
from discogsUtils import datpiffUtils

## RockCorner
from artistRC import artistRC
from artistsRC import artistsRC
from discogsUtils import rockcornerUtils

## CDandLP
from artistCL import artistCL
from artistsCL import artistsCL
from discogsUtils import cdandlpUtils

## MusicStack
from artistMS import artistMS
from artistsMS import artistsMS
from discogsUtils import musicstackUtils

## MetalStorm
from artistMT import artistMT
from artistsMT import artistsMT
from discogsUtils import metalstormUtils



import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-11 19:16:04.098161


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (All Music)

In [ ]:
for i in range(100):
    artsAM.parseArtistModValFiles(i, force=True)

In [ ]:
artMB = artistMB(discMB)
retval = artMB.getData("/Volumes/Biggy/Discog/artists-musicbrainz/0/148150553282083715092252970758494088400.p")
retval.url.get()
retval.ID.get()

# Parse Artists (Parallel)

In [27]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artistsDC(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

discDP   = discogs('datpiff')
artsDP   = artistsDP(discDP)
dutilsDP = datpiffUtils()

discRC   = discogs('rockcorner')
artsRC   = artistsRC(discRC)
dutilsRC = rockcornerUtils()

discCL   = discogs('cdandlp')
artsCL   = artistsCL(discCL)
dutilsCL = cdandlpUtils()

discMS   = discogs('musicstack')
artsMS   = artistsMS(discMS)
dutilsMS = musicstackUtils()

discMT   = discogs('metalstorm')
artsMT   = artistsMT(discMT)
dutilsMT = metalstormUtils()


def parseArtistsAM(modVal, force=False, doExtra=False):
    artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=False, doExtra=False):
    artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    artsMB.parseArtistModValFiles(modVal, force=True)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    artsRC.parseArtistModValFiles(modVal, force=True)

def parseArtistsCL(modVal, force=False, doExtra=False):
    artsCL.parseArtistModValFiles(modVal, force=True)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=True)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#artsLM.parseArtistModValFiles(0, force=True)

In [ ]:
#getFile("/Volumes/Biggy/Discog/artists-lastfm-db/metadata/0-MediaMetadata.p")

In [23]:
#artsDC.parseArtistModValFiles(29, force=True)
#arts.parseArtistModValFiles(50, force=True)
artsCL.parseArtistModValFiles(25)

Parsing Artist Files For ModVal 25
  ===> Found 8 files. There are 1 new files to parse.
removeFile("/Volumes/Biggy/Discog/artists-cdandlp/25/83536884883025.p")
Saving 1 new artist IDs to /Volumes/Biggy/Discog/artists-cdandlp-db/25-DB.p
  --> This file is 4.2kB.
Saving 7 new artist IDs name data to /Volumes/Biggy/Discog/artists-cdandlp-db/metadata/25-Metadata.p
  --> This file is 499B.
Saving 7 new artist IDs media data to /Volumes/Biggy/Discog/artists-cdandlp-db/metadata/25-MediaMetadata.p
  --> This file is 2.5kB.


1

In [ ]:
ifile="/Volumes/Biggy/Discog/artists-lastfm/0/53902704000.p"
bsdata = getHTML(ifile)
artLM = artistLM()
#artLM.getData(ifile)

In [5]:
artCL  = artistCL()
#retval = artCL.getData("/Volumes/Biggy/Discog/artists-cdandlp/50/30468865745050.p")

In [ ]:
retval.artist.get()

In [ ]:
retval.ID.get()

In [ ]:
retval.url.get()

In [6]:
from glob import glob
files = glob("/Volumes/Biggy/Discog/artists-cdandlp/*/*.p")
errs = []
for ifile in files:
    try:
        retval = artCL.getData(ifile)
        ID = retval.ID.ID
        #print(ifile,'\t\t',ID == dutilsCL.getArtistID(url=retval.url.url))
    except:
        print("Err: {0}".format(ifile))
        errs.append(ifile)

Err: /Volumes/Biggy/Discog/artists-cdandlp/11/12864696649811.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/11/62452119003011.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/11/7103524999511.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/11/84142929101611.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/11/84727055536611.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/11/9533739215511.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/12/21985706660012.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/12/36880586704912.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/12/42862374131712.p
Err: /Volumes/Biggy/Discog/artists-cdandlp/12/55562260672212.p


In [ ]:
from fsUtils import removeFile
for err in errs:
    removeFile(err)

In [ ]:
artCL = artistCL()
retval = artCL.getData("/Volumes/Biggy/Discog/artists-cdandlp/37/6521810621937.p")
#retval = artCL.getData("/Volumes/Biggy/Discog/artists-cdandlp/16/33212050046916.p")
retval.get()
print(retval.artist.get())
print(retval.url.get())
print(retval.ID.get())
print(dutilsCL.getArtistID(retval.url.url))

In [28]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
#parseArtistsParallel(db="Discogs", nProcs=3)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=4)
#parseArtistsParallel(db="LastFM", nProcs=3)
parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Sun Apr 12, 2020 12:47:24 for Parsing All Artists
Current Time is Sun Apr 12, 2020 12:47:24 for Parallel Artist Parsing
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 18
Parsing Artist Files For ModVal 9
Forcing Reloads of ModVal=0
Forcing Reloads of ModVal=18
Forcing Reloads of ModVal=9
  Processing 9 files.
  Processing 19 files.
  Processing 14 files.
	Processed 0/14 files.
	Processed 0/19 files.
	Processed 0/9 files.
Saving 9 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/18-DB.p
  --> This file is 31.1kB.
Saving 9 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/18-Metadata.p
  --> This file is 461B.
Saving 9 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/18-MediaMetadata.p
  --> This file is 24.2kB.
Parsing Artist Files For ModVal 19
Forcing Reloads of ModVal=19
  Processing 18 files.
	Processed 0/18 files.
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-

  --> This file is 44.5kB.
Saving 10 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/14-Metadata.p
  --> This file is 493B.
Saving 10 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/14-MediaMetadata.p
  --> This file is 35.4kB.
Parsing Artist Files For ModVal 15
Forcing Reloads of ModVal=15
  Processing 11 files.
	Processed 0/11 files.
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/5-DB.p
  --> This file is 58.7kB.
Saving 14 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/5-Metadata.p
  --> This file is 654B.
Saving 14 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/5-MediaMetadata.p
Saving 14 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/24-DB.p
  --> This file is 45.8kB.
Parsing Artist Files For ModVal 6
Forcing Reloads of ModVal=6
  Processing 11 files.
	Processed 0/11 files.
  --> This file is 55.0kB.
Saving 1

  --> This file is 33.4kB.
Saving 9 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/38-Metadata.p
  --> This file is 443B.
Saving 9 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/38-MediaMetadata.p
  --> This file is 26.1kB.
Parsing Artist Files For ModVal 39
Forcing Reloads of ModVal=39
  Processing 11 files.
	Processed 0/11 files.
Saving 17 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/29-DB.p
Saving 11 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/39-DB.p
  --> This file is 68.7kB.
Saving 17 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/29-Metadata.p
  --> This file is 776B.
Saving 17 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/29-MediaMetadata.p
  --> This file is 43.3kB.
Saving 11 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/39-Metadata.p
  --> This file is 536B.
Saving 11 ne

  --> This file is 33.8kB.
Saving 8 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/44-Metadata.p
  --> This file is 393B.
Saving 8 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/44-MediaMetadata.p
  --> This file is 26.3kB.
Parsing Artist Files For ModVal 54
Forcing Reloads of ModVal=54
  Processing 13 files.
	Processed 0/13 files.
Saving 19 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/34-DB.p
  --> This file is 80.5kB.
Saving 19 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/34-Metadata.p
  --> This file is 842B.
Saving 19 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/34-MediaMetadata.p
  --> This file is 63.9kB.
Parsing Artist Files For ModVal 35
Forcing Reloads of ModVal=35
  Processing 19 files.
	Processed 0/19 files.
Saving 10 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/53-DB.p
  --> This file is 36.9kB.
Savin

Parsing Artist Files For ModVal 68
Forcing Reloads of ModVal=68
  Processing 13 files.
	Processed 0/13 files.
  --> This file is 55.7kB.
Saving 13 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/59-Metadata.p
  --> This file is 704B.
Saving 13 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/59-MediaMetadata.p
  --> This file is 48.5kB.
Saving 13 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/75-Metadata.p
  --> This file is 640B.
Saving 13 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/75-MediaMetadata.p
  --> This file is 44.0kB.
Parsing Artist Files For ModVal 60
Forcing Reloads of ModVal=60
  Processing 16 files.
	Processed 0/16 files.
  --> This file is 38.1kB.
Parsing Artist Files For ModVal 76
Forcing Reloads of ModVal=76
  Processing 15 files.
	Processed 0/15 files.
Saving 13 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/68-DB.p


  --> This file is 482B.
Saving 9 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/82-MediaMetadata.p
  --> This file is 26.5kB.
  --> This file is 27.1kB.
Parsing Artist Files For ModVal 83
Forcing Reloads of ModVal=83
  Processing 11 files.
	Processed 0/11 files.
Saving 10 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/91-DB.p
  --> This file is 33.8kB.
Saving 10 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/91-Metadata.p
  --> This file is 490B.
Saving 10 new artist IDs media data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/91-MediaMetadata.p
  --> This file is 26.2kB.
Parsing Artist Files For ModVal 92
Forcing Reloads of ModVal=92
  Processing 12 files.
	Processed 0/12 files.
Saving 11 new artist IDs to /Volumes/Biggy/Discog/artists-rockcorner-db/83-DB.p
  --> This file is 40.9kB.
Saving 11 new artist IDs name data to /Volumes/Biggy/Discog/artists-rockcorner-db/metadata/83-Metadata.p
  --

In [9]:
for err in errs:
    removeFile(err)

In [7]:
from glob import glob
errs = []
from fsUtils import moveFile, removeFile
files=glob("/Volumes/Biggy/Discog/artists-cdandlp/*/*.p")
for ifile in files:
    retval = artCL.getData(ifile)
    url    = retval.url.url
    ID     = retval.ID.ID
    #print(ifile,'\t',ID,'\t',dutilsCL.getArtistID(url))
    savename = artsCL.getArtistSavename(ID)
    if ifile == savename:
        continue
    moveFile(src=ifile,dst=savename, debug=True)
    errs.append(ifile)

Moving /Volumes/Biggy/Discog/artists-cdandlp/26/64526040644026.p to /Volumes/Biggy/Discog/artists-cdandlp/44/46011250269544.p
  Not moving because /Volumes/Biggy/Discog/artists-cdandlp/44/46011250269544.p exists.
Moving /Volumes/Biggy/Discog/artists-cdandlp/3/70794261073903.p to /Volumes/Biggy/Discog/artists-cdandlp/0/14949541170200.p
  Not moving because /Volumes/Biggy/Discog/artists-cdandlp/0/14949541170200.p exists.
Moving /Volumes/Biggy/Discog/artists-cdandlp/3/82599934909803.p to /Volumes/Biggy/Discog/artists-cdandlp/97/78353573040297.p
  Not moving because /Volumes/Biggy/Discog/artists-cdandlp/97/78353573040297.p exists.
Moving /Volumes/Biggy/Discog/artists-cdandlp/37/6521810621937.p to /Volumes/Biggy/Discog/artists-cdandlp/80/58265960848480.p
  Not moving because /Volumes/Biggy/Discog/artists-cdandlp/80/58265960848480.p exists.
Moving /Volumes/Biggy/Discog/artists-cdandlp/41/57288206697041.p to /Volumes/Biggy/Discog/artists-cdandlp/80/58265960848480.p
  Not moving because /Volum

In [7]:
artists=["Almanac", 
"Big Country",
"Bobby Charles",
"Go-A",
"Hailey Whitters",
"John Waite",
"Pengshui",
"Throbbing Gristle"]
for artist in artists[1:]:
    artsAM.searchAllMusicForArtist(artist)
    artsDC.searchDiscogForArtist(artist)
    artsMB.searchMusicBrainzForArtist(artist)
    #discRM   = discogs('rateyourmusic')
    artsLM.searchLastFMForArtist(artist)
    artsRC.searchRockCornerForArtist(artist)
    artsCL.searchCDandLPForArtist(artist)
    #artsMS.searchMusicStackForArtist(artist)



===================== Searching For Big Country =====================
Found 20 artists
1 / 20 	: 0000056625 	 https://www.allmusic.com/artist/big-country-mn0000056625/discography/all
2 / 20 	: 0002016355 	 https://www.allmusic.com/artist/big-country-mn0002016355/discography/all
3 / 20 	: 0000759580 	 https://www.allmusic.com/artist/big-country-bluegrass-mn0000759580/discography/all
4 / 20 	: 0001808151 	 https://www.allmusic.com/artist/tony-big-country-addams-mn0001808151/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/51/0001808151.p (force=False)
  --> This file is 14.3kB.
Done. Sleeping for 2 seconds
5 / 20 	: 0001837543 	 https://www.allmusic.com/artist/stuart-adamsons-big-country-mn0001837543/discography/all
Saving /Volumes/Biggy/Discog/artists-allmusic/43/0001837543.p (force=False)
  --> This file is 14.3kB.
Done. Sleeping for 2 seconds
6 / 20 	: 0000302352 	 https://www.allmusic.com/artist/karl-shiflett-the-big-country-show-mn0000302352/discography/all
7 / 20 	: 

33 / 251 	: 7 	 https://www.discogs.com/artist/1162454-Yea-Big-Kid-Static?sort=year%2Casc&limit=500
34 / 251 	: 7 	 https://www.discogs.com/artist/3886054-The-Big-Iron?sort=year%2Casc&limit=500
35 / 251 	: 7 	 https://www.discogs.com/artist/3983895-Big-Tubba-Mista?sort=year%2Casc&limit=500
36 / 251 	: 7 	 https://www.discogs.com/artist/6609543-No-Big-Deal-3?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/43/6609543.p (force=False)
  --> This file is 16.6kB.
Done. Sleeping for 2 seconds
37 / 251 	: 7 	 https://www.discogs.com/artist/5375556-Big-Kountry?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/56/5375556.p (force=False)
  --> This file is 15.4kB.
Done. Sleeping for 2 seconds
38 / 251 	: 7 	 https://www.discogs.com/artist/3899195-Jerry-Teel-And-The-Big-City-Stompers?sort=year%2Casc&limit=500
39 / 251 	: 7 	 https://www.discogs.com/artist/5831884-Des-Wilson-The-Big-Country?sor

82 / 251 	: 7 	 https://www.discogs.com/artist/2225107-Eddy-Polo?sort=year%2Casc&limit=500
83 / 251 	: 6 	 https://www.discogs.com/artist/297871-Frank-Simon?sort=year%2Casc&limit=500
84 / 251 	: 7 	 https://www.discogs.com/artist/2808478-Adrian-Slattery?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/78/2808478.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
85 / 251 	: 7 	 https://www.discogs.com/artist/1264005-Eddie-Polo?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/5/1264005.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
86 / 251 	: 6 	 https://www.discogs.com/artist/343045-Curly-Smith?sort=year%2Casc&limit=500
87 / 251 	: 6 	 https://www.discogs.com/artist/416193-John-Rich?sort=year%2Casc&limit=500
88 / 251 	: 7 	 https://www.discogs.com/artist/6964709-Craig-Dermer?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Savi

136 / 251 	: 6 	 https://www.discogs.com/artist/317479-Lady-N?sort=year%2Casc&limit=500
137 / 251 	: 6 	 https://www.discogs.com/artist/706522-Tim-James-2?sort=year%2Casc&limit=500
138 / 251 	: 6 	 https://www.discogs.com/artist/296493-Jack-Hammer?sort=year%2Casc&limit=500
139 / 251 	: 7 	 https://www.discogs.com/artist/4066387-Jack-Wheaton?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/87/4066387.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
140 / 251 	: 7 	 https://www.discogs.com/artist/1726290-Tom-Stern?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/90/1726290.p (force=False)
  --> This file is 22.1kB.
Done. Sleeping for 2 seconds
141 / 251 	: 7 	 https://www.discogs.com/artist/2556884-La-Fawn-Paul?sort=year%2Casc&limit=500
142 / 251 	: 7 	 https://www.discogs.com/artist/1003105-Kent-Heckman?sort=year%2Casc&limit=500
143 / 251 	: 6 	 https://www.di

195 / 251 	: 7 	 https://www.discogs.com/artist/3240600-Dexi-3?sort=year%2Casc&limit=500
196 / 251 	: 7 	 https://www.discogs.com/artist/7406273-Billy-Johnson-16?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/73/7406273.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
197 / 251 	: 7 	 https://www.discogs.com/artist/1408046-Ella-Mental?sort=year%2Casc&limit=500
198 / 251 	: 6 	 https://www.discogs.com/artist/708963-The-Scroungers?sort=year%2Casc&limit=500
199 / 251 	: 7 	 https://www.discogs.com/artist/1890653-The-Unplayable-Sofa-Guitar?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/53/1890653.p (force=False)
  --> This file is 16.9kB.
Done. Sleeping for 2 seconds
200 / 251 	: 7 	 https://www.discogs.com/artist/3303574-Paul%C3%A3o-Sakae-Tahira?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/74/3303574.p

...

Saving /Volumes/Biggy/Discog/artists-cdandlp/45/57240901717245.p (force=False)
  --> This file is 37.0kB.
Done. Sleeping for 2 seconds
2 / 7 	: 27962339377189 	 https://www.cdandlp.com/en/john-waite/artist 	 /Volumes/Biggy/Discog/artists-cdandlp/89/27962339377189.p
Saving /Volumes/Biggy/Discog/artists-cdandlp/89/27962339377189.p (force=False)
  --> This file is 55.3kB.
Done. Sleeping for 2 seconds
3 / 7 	: 88968212285408 	 https://www.cdandlp.com/en/john-waite-missing-you-12/artist 	 /Volumes/Biggy/Discog/artists-cdandlp/8/88968212285408.p
Saving /Volumes/Biggy/Discog/artists-cdandlp/8/88968212285408.p (force=False)
  --> This file is 37.1kB.
Done. Sleeping for 2 seconds
4 / 7 	: 63752026795668 	 https://www.cdandlp.com/en/nazareth-scorpions-leo-sayer-10cc-john-waite/artist 	 /Volumes/Biggy/Discog/artists-cdandlp/68/63752026795668.p
Saving /Volumes/Biggy/Discog/artists-cdandlp/68/63752026795668.p (force=False)
  --> This file is 37.2kB.
Done. Sleeping for 2 seconds
5 / 7 	: 95809865033

Saving /Volumes/Biggy/Discog/artists-allmusic/52/0003247852.p (force=False)
  --> This file is 14.8kB.
Done. Sleeping for 2 seconds


===================== Searching For Throbbing Gristle =====================
Found 91 artists
2 / 91 	: 5 	 https://www.discogs.com/artist/12589-Throbbing-Gristle?sort=year%2Casc&limit=500
3 / 91 	: 7 	 https://www.discogs.com/artist/4184798-Throbbing-Wafle?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/98/4184798.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
4 / 91 	: 6 	 https://www.discogs.com/artist/778177-SinclairBrooks?sort=year%2Casc&limit=500
5 / 91 	: 5 	 https://www.discogs.com/artist/57533-Monte-Cazazza?sort=year%2Casc&limit=500
6 / 91 	: 7 	 https://www.discogs.com/artist/2020530-X-TG?sort=year%2Casc&limit=500
7 / 91 	: 6 	 https://www.discogs.com/artist/524303-Neville-Brody?sort=year%2Casc&limit=500
8 / 91 	: 6 	 https://www.discogs.com/artist/229592-Val-Denham?s

  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/16/563216.p (force=False)
  --> This file is 16.0kB.
Done. Sleeping for 2 seconds
66 / 91 	: 5 	 https://www.discogs.com/artist/10210-Exotica?sort=year%2Casc&limit=500
67 / 91 	: 7 	 https://www.discogs.com/artist/1872730-Life-In-Cage?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Biggy/Discog/artists-discogs/30/1872730.p (force=False)
  --> This file is 19.6kB.
Done. Sleeping for 2 seconds
68 / 91 	: 7 	 https://www.discogs.com/artist/1011032-Intolerance?sort=year%2Casc&limit=500
69 / 91 	: 7 	 https://www.discogs.com/artist/5500802-A-Million-Machines?sort=year%2Casc&limit=500
70 / 91 	: 5 	 https://www.discogs.com/artist/21148-23-Skidoo?sort=year%2Casc&limit=500
71 / 91 	: 7 	 https://www.discogs.com/artist/1116830-%E3%82%B5%E3%82%A4%E3%82%B1%E3%82%A2%E3%82%A6%E3%83%84?sort=year%2Casc&limit=500
72 / 91 	: 6 	 https://www.discogs.com/artist/186586-AA23?sort=year%2Casc&limit=500
73 / 9

16 / 30 	: 19131117705 	 https://www.last.fm/music/Throbbing%2BGristle%2B%25252B%2BScanner/+albums?order=release_date


HTTPError: HTTP Error 404: NOT FOUND

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

In [ ]:
from fsUtils import removeFile